<a href="https://www.kaggle.com/code/pawankumaragrawal/green-energy-forecast?scriptVersionId=112009549" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/forecast-green-energy/test_WudNWDM.csv
/kaggle/input/forecast-green-energy/train_IxoE5JN.csv


In [2]:
!pip install catboost
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [3]:
df_train = pd.read_csv('/kaggle/input/forecast-green-energy/train_IxoE5JN.csv')
df_train.head(3)

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    93092 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [5]:
df_train['energy'] = df_train['energy'].fillna(method='ffill')
df_train.head(3)

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500


In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   row_id    94992 non-null  int64  
 1   datetime  94992 non-null  object 
 2   energy    94992 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [7]:
df_train.rename(columns = {'datetime':'date_time'}, inplace = True)
df_train.head(3)

,row_id,date_time,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500


In [8]:
df_train['year'] = df_train['date_time'].str[:4]
df_train['year'] = df_train['year'].astype('int64')

df_train['month'] = df_train['date_time'].str[5:7]
df_train['month'] = df_train['month'].astype('int64')

df_train['day'] = df_train['date_time'].str[8:10]
df_train['day'] = df_train['day'].astype('int64')

df_train['hour'] = df_train['date_time'].str[11:13]
df_train['hour'] = df_train['hour'].astype('int64')

df_train.head(3)

,row_id,date_time,energy,year,month,day,hour
0,1,2008-03-01 00:00:00,1259.985563,2008,3,1,0
1,2,2008-03-01 01:00:00,1095.541500,2008,3,1,1
2,3,2008-03-01 02:00:00,1056.247500,2008,3,1,2


In [9]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   row_id     94992 non-null  int64  
 1   date_time  94992 non-null  object 
 2   energy     94992 non-null  float64
 3   year       94992 non-null  int64  
 4   month      94992 non-null  int64  
 5   day        94992 non-null  int64  
 6   hour       94992 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 5.1+ MB


In [10]:
df_train['date_time'] = pd.to_datetime(df_train['date_time'])

In [11]:
df_train['weekday'] = df_train['date_time'].dt.dayofweek
df_train.drop(['date_time', 'row_id'], axis = 1, inplace = True)
df_train.head(3)

,energy,year,month,day,hour,weekday
0,1259.985563,2008,3,1,0,5
1,1095.541500,2008,3,1,1,5
2,1056.247500,2008,3,1,2,5


In [12]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94992 entries, 0 to 94991
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   energy   94992 non-null  float64
 1   year     94992 non-null  int64  
 2   month    94992 non-null  int64  
 3   day      94992 non-null  int64  
 4   hour     94992 non-null  int64  
 5   weekday  94992 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 4.3 MB


In [13]:
from sklearn.model_selection import train_test_split

X = df_train.drop(columns = 'energy', axis = 1)
y = df_train['energy']

(X_train, X_val, y_train, y_val) = train_test_split(X, y, test_size = 0.1)

In [14]:
X_train.head(3)

,year,month,day,hour,weekday
52614,2014,3,2,6,6
46088,2013,6,3,8,0
38758,2012,8,1,22,2


In [15]:
X_val.head(3)

,year,month,day,hour,weekday
63354,2015,5,23,18,5
20919,2010,7,20,15,1
31330,2011,9,27,10,1


In [16]:
from catboost import CatBoostRegressor as cbr

cat = cbr(iterations=3000, learning_rate=0.01, depth=7, l2_leaf_reg= 3, loss_function='RMSE') 

catreg = cat.fit(X_train,y_train,verbose_eval=200,plot=True,eval_set=(X_val,y_val))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 367.1525100	test: 364.4098251	best: 364.4098251 (0)	total: 69.7ms	remaining: 3m 29s
200:	learn: 187.6280718	test: 186.1949457	best: 186.1949457 (200)	total: 1.98s	remaining: 27.5s
400:	learn: 160.1241334	test: 159.3824696	best: 159.3824696 (400)	total: 3.87s	remaining: 25.1s
600:	learn: 148.0313682	test: 147.5793767	best: 147.5793767 (600)	total: 5.75s	remaining: 22.9s
800:	learn: 139.9881613	test: 139.8112626	best: 139.8112626 (800)	total: 7.62s	remaining: 20.9s
1000:	learn: 134.0287709	test: 134.1201443	best: 134.1201443 (1000)	total: 9.55s	remaining: 19.1s
1200:	learn: 129.5161892	test: 129.8661129	best: 129.8661129 (1200)	total: 11.5s	remaining: 17.3s
1400:	learn: 125.4949481	test: 126.1362080	best: 126.1362080 (1400)	total: 13.5s	remaining: 15.4s
1600:	learn: 122.0050018	test: 122.9104435	best: 122.9104435 (1600)	total: 15.4s	remaining: 13.5s
1800:	learn: 119.1078162	test: 120.1403057	best: 120.1403057 (1800)	total: 17.4s	remaining: 11.6s
2000:	learn: 116.4793822	test: 1

In [17]:
df_test = pd.read_csv('/kaggle/input/forecast-green-energy/test_WudNWDM.csv' )
df_test.head(3)

,row_id,datetime
0,94993,2019-01-01 00:00:00
1,94994,2019-01-01 01:00:00
2,94995,2019-01-01 02:00:00


In [18]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   row_id    26304 non-null  int64 
 1   datetime  26304 non-null  object
dtypes: int64(1), object(1)
memory usage: 411.1+ KB


In [19]:
df_test.rename(columns = {'datetime':'date_time'}, inplace = True)
df_test.head(3)

,row_id,date_time
0,94993,2019-01-01 00:00:00
1,94994,2019-01-01 01:00:00
2,94995,2019-01-01 02:00:00


In [20]:
df_test['year'] = df_test['date_time'].str[:4]
df_test['year'] = df_test['year'].astype('int64')

df_test['month'] = df_test['date_time'].str[5:7]
df_test['month'] = df_test['month'].astype('int64')

df_test['day'] = df_test['date_time'].str[8:10]
df_test['day'] = df_test['day'].astype('int64')

df_test['hour'] = df_test['date_time'].str[11:13]
df_test['hour'] = df_test['hour'].astype('int64')

df_test.head(3)

,row_id,date_time,year,month,day,hour
0,94993,2019-01-01 00:00:00,2019,1,1,0
1,94994,2019-01-01 01:00:00,2019,1,1,1
2,94995,2019-01-01 02:00:00,2019,1,1,2


In [21]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   row_id     26304 non-null  int64 
 1   date_time  26304 non-null  object
 2   year       26304 non-null  int64 
 3   month      26304 non-null  int64 
 4   day        26304 non-null  int64 
 5   hour       26304 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 1.2+ MB


In [22]:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])

In [23]:
df_test['weekday'] = df_test['date_time'].dt.dayofweek
df_test2 = df_test.drop(['date_time', 'row_id'], axis = 1)
df_test2.head(3)

,year,month,day,hour,weekday
0,2019,1,1,0,1
1,2019,1,1,1,1
2,2019,1,1,2,1


In [24]:
df_test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26304 entries, 0 to 26303
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   year     26304 non-null  int64
 1   month    26304 non-null  int64
 2   day      26304 non-null  int64
 3   hour     26304 non-null  int64
 4   weekday  26304 non-null  int64
dtypes: int64(5)
memory usage: 1.0 MB


In [25]:
# performing predictions on the test dataset
X_test=df_test2.to_numpy()
y_pred_test = catreg.predict(X_test)

In [26]:
y_pred_test

array([1743.4998724 , 1630.70595435, 1552.6504989 , ..., 2430.17010631,
       2381.42735845, 2215.78077076])

In [27]:
df_energy = pd.DataFrame(y_pred_test, columns =['energy'])
df_energy.head(3)

,energy
0,1743.499872
1,1630.705954
2,1552.650499


In [28]:
df_row_id = df_test.drop(['date_time', 'year', 'month', 'day', 'hour', 'weekday'], axis = 1)
df_row_id.head(3)

,row_id
0,94993
1,94994
2,94995


In [29]:
df_concatenated = pd.concat([df_row_id, df_energy], axis=1)
df_submission = df_concatenated
df_submission

,row_id,energy
0,94993,1743.499872
1,94994,1630.705954
2,94995,1552.650499
3,94996,1496.021775
4,94997,1485.259432
...,...,...
26299,121292,2525.184515
26300,121293,2470.658418
26301,121294,2430.170106
26302,121295,2381.427358


In [30]:
df_submission.to_csv('submission.csv')